In [8]:
%pip install geopandas


     |████████████████████████████████| 323 kB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 65.5 MB/s eta 0:00:01
     |████████████████████████████████| 10.8 MB 77.2 MB/s eta 0:00:01
     |████████████████████████████████| 19.5 MB 38.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 79.9 MB/s eta 0:00:01
     |████████████████████████████████| 5.3 MB 73.9 MB/s eta 0:00:01
     |████████████████████████████████| 509 kB 95.2 MB/s eta 0:00:01
     |████████████████████████████████| 347 kB 83.7 MB/s eta 0:00:01
     |████████████████████████████████| 159 kB 90.4 MB/s eta 0:00:01
You should consider upgrading via the '/Users/moriseiitsu/Dropbox/IDEAMAPS/ideamaps-models/models/emergency-maternal-care/scripts/.venv_requirements_test/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Set paths to access data
# Define directories
data_inputs = './Nairobi/data-inputs/'
data_temp = './Nairobi/data-temp/'
data_outputs = '../nairobi/'

In [9]:
import geopandas as gpd
import os
healthcare_facilities_validated = gpd.read_file(data_inputs + 'helthcare-facilities.geojson')

In [10]:
healthcare_facilities_validated

,fid,field_1_1,Admin1,Facility_n,Facility_t,Ownership,Lat,Long,LL_source,Ward,Sub County,Level,Offer EmOC (Y/N),Type of EmOC (basic/comprehensive),Total In-patient beds,Maternity beds,Maternity theatres,Operating time,Comments,geometry
0,1,Kenya,Nairobi,Dandora II Health Centre,Health Centre,Local authority,-1.257610,36.887750,GPS,Dandora Area III,Embakasi North,Level 3,True,BMOC,15,N/A,N/A,24hrs,None,POINT (36.88775 -1.25761)
1,2,Kenya,Nairobi,Embakasi Health Centre,Health Centre,Local authority,-1.306200,36.914700,GPS,Embakasi,Embakasi East,Level 3,True,CeMOC & BMOC,N/A,N/A,N/A,24hrs,None,POINT (36.9147 -1.3062)
2,3,Kenya,Nairobi,Imara Health Centre,Health Centre,FBO,-1.324300,36.888700,Google Earth,Kware,Embakasi South,Level 3,True,BMOC,6,N/A,N/A,24hrs & weekends,None,POINT (36.8887 -1.3243)
3,4,Kenya,Nairobi,Jonalifa Clinic,Clinic,NGO,-1.285900,36.739880,Google Earth,Kawangware,Dagoretti South,Level 2,True,BMOC,8,N/A,N/A,24hrs & weekends,None,POINT (36.73988 -1.2859)
4,5,Kenya,Nairobi,Kangemi Health Centre,Health Centre,Local authority,-1.264600,36.749300,GPS,Kitisuru,Westlands,Level 3,True,BMOC,N/A,N/A,N/A,24hrs,None,POINT (36.7493 -1.2646)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,150,Kenya,Nairobi,Nairobi Hospital,Comprehensive Teaching & Tertiary Referral Hos...,Private,-1.294743,36.800247,KMHFR,Kilimani,Dagoretti North,Level 5,True,CeMOC&BMOC,350,N/A,N/A,24hrs+weekends,None,POINT (36.80025 -1.29474)
149,151,Kenya,Nairobi,Al-Basrah Hospital Limited,Primary care hospitals,Private,-1.255881,36.876857,KMHFR,Kiamaiko,Mathare,Level 4,True,BMOC,N/A,10,1,24hrs+weekends,None,POINT (36.87686 -1.25588)
150,152,Kenya,Nairobi,St. Oswald Medical Centre,Medical Center,Private,-1.290367,36.729885,KMHFR,Riruta,Dagoretti South,Level 3,True,BMOC,12,2,N/A,24hrs+weekends,None,POINT (36.72989 -1.29037)
151,153,Kenya,Nairobi,Nairobi West Hospital,Primary Care hospitals,Private,-1.306840,36.825760,KMHFR,Nairobi West,Langata,Level 4,True,BMOC,152,N/A,N/A,24hrs+weekends,None,POINT (36.82576 -1.30684)


In [1]:
import numpy as np

In [11]:
conditions = [
    healthcare_facilities_validated['specific_owner'].isin([1, 3, 6]),
    healthcare_facilities_validated['specific_owner'].isin([2, 4, 5])
]
choices = ['Public Comprehensive EmOC', 'Private Comprehensive EmOC']

# Explicitly set the default to a string
healthcare_facilities_validated['Validation'] = np.select(
    conditions, choices, default='Other'
)

KeyError: 'specific_owner'

In [ ]:
study_area = gpd.read_file(data_inputs + '100mGrid.gpkg')
raster_path = data_inputs + 'nga_f_15_49_2015_1km.tif'

In [2]:
pip install rasterio
import rasterio
from rasterio.mask import mask

ModuleNotFoundError: No module named 'rasterio'

In [ ]:
with rasterio.open(raster_path) as dataset:
    geometries = [study_area.geometry.unary_union.__geo_interface__]
    clipped_image, clipped_transform = mask(dataset, geometries, crop=True)
    band1 = clipped_image[0] # Read the first band of the raster

In [ ]:
out_meta = dataset.meta.copy()
out_meta.update({
        "height": clipped_image.shape[1],
        "width": clipped_image.shape[2],
        "transform": clipped_transform
    })

In [ ]:
with rasterio.open(data_inputs + 'nairobi_nga_f_15_49_2015_1km.tif', "w", **out_meta) as dest:
    dest.write(clipped_image)

In [ ]:
rows, cols = np.where(band1 > 0)
grid_cells = [clipped_transform * (col + 0.5, row + 0.5) for row, col in zip(rows, cols)]
population_values = band1[rows, cols]

In [3]:
from shapely.geometry import Point
import pandas as pd

In [ ]:
grid_df = pd.DataFrame(grid_cells, columns=["longitude", "latitude"])
grid_df["population"] = population_values

grid_df["grid_code"] = range(len(grid_df))
population_centroids_gdf = gpd.GeoDataFrame(grid_df, geometry=[Point(xy) for xy in zip(grid_df["longitude"], grid_df["latitude"])])
population_centroids_gdf.set_crs("EPSG:4326", inplace=True)

population_centroids_gdf.to_file(data_temp + "population_centroids.gpkg", driver="GPKG")